# Работа с базой данных

- Сессии и фабрики сессий: как управлять сессиями для взаимодействия с базой данных и как применять их через декораторы.

- Добавление данных в таблицы: разберём безопасные методы добавления записей с использованием ORM. Также обсудим метод flush и разницу между ним и commit.

- Извлечение данных из таблиц: большой блок, в котором научимся извлекать данные через select, используя фильтры (например, where, filter, filter_by). Также обсудим работу с «грязными» данными и преобразование объектов SQLAlchemy в удобные словари Python с помощью Pydantic. В этом блоке разберём и методы SQLAlchemy, такие как scalar, scalars, scalar_one_or_none, all и другие.

# Что такое сессия?
Сессия в SQLAlchemy — это основной инструмент для взаимодействия с базой данных. Представьте её как рабочую область, где происходят все операции: добавление, удаление, извлечение, обновление данных. Все запросы к базе данных выполняются через сессию, без неё никакие операции невозможны.

Сессия управляет транзакциями и следит за состоянием объектов, с которыми вы работаете. Она не устанавливает прямого соединения с базой, а абстрагирует этот процесс. Все изменения отправляются в базу данных через метод commit(). В случае ошибки их можно отменить с помощью rollback().

# Фабрика сессий
Фабрика сессий — это специальная функция для создания новых сессий по мере необходимости. В SQLAlchemy это реализуется с помощью sessionmaker(). Этот объект создаёт сессии, которые можно использовать для работы с базой данных.

sync реализация

In [ ]:
from sqlalchemy.orm import sessionmaker

...
engine = ...
session_maker = sessionmaker(engine, expire_on_commit=False)  # Фабрика сессий



def connection(method):
    def wrapper(*args, **kwargs):
        with session_maker() as session:
            try:
                return method(*args, session=session, **kwargs)
            except Exception as e:
                session.rollback()
                raise e
            finally:
                session.close()

    return wrapper


async реализация

In [ ]:

from sqlalchemy.ext.asyncio import async_sessionmaker, create_async_engine


...
engine = create_async_engine(...)
session_maker = async_sessionmaker(engine, expire_on_commit=False)  # Фабрика сессий

def connection(method):
    async def wrapper(*args, **kwargs):
        async with session_maker() as session:
            try:
                return await method(*args, session=session, **kwargs)
            except Exception as e:
                await session.rollback()
                raise e
            finally:
                await session.close()
    return wrapper


# Создание декоратора
для работы с сессией по мере необходимости

То есть, для того чтоб начать взаимодействовать с базой данных (например для получения или добавления туда информации) нам всегда необходимо быть в рамках сессии. Вопрос только в том какое количество операций вы будете выполнять в рамках одной такой сессии, до ее закрытия.

Основные подходы к управлению сессиями:

- Открытие сессии на каждое действие: для каждого действия с базой данных создаётся новая сессия. Этот подход эффективен для небольших проектов, но на крупных проектах он может привести к дополнительным накладным расходам.

- Открытие сессии на весь блок операций: сессия создаётся один раз перед серией операций и закрывается по завершению всех действий. Это позволяет объединить несколько запросов в одну сессию, что экономит ресурсы и повышает производительность.

In [ ]:
# sync
def connection(method):
    def wrapper(*args, **kwargs):
        with session_maker() as session:
            try:
                return method(*args, session = session, **kwargs)
            except Exception as e:
                session.rollback()
                raise e
            finally:
                session.close()
    return wrapper

# async
def connection(method):
    async def wrapper(*args, **kwargs):
        async with async_session_maker() as session:
            try:
                return await method(*args, session = session, **kwargs)
            except Exception as e:
                await session.rollback()
                raise e
            finally:
                await  session.close()
    return wrapper

Как работает этот декоратор:

connection принимает исходную функцию для обёртки.

wrapper — это функция-обёртка, которая принимает все аргументы исходной функции.

async with async_session_maker() автоматически создаёт и закрывает сессию в асинхронном режиме, освобождая вас от необходимости управлять сессией вручную.

Сессия передаётся в исходную функцию через аргумент session.

В случае ошибки выполняется откат транзакции через rollback(), а затем сессия закрывается.

Создадим функцию добавления пользователя в базу

In [ ]:
@classmethod
@connection
def create(
    cls,
    username: str,
    password: str,
    email: str,
    phone=None,
    session: Session = None,
    **data,
):
    new_user = cls(username=username, password=password, email=email, phone=phone)
    session.add(new_user)
    session.commit()

    new_profile = Profile(**data)
    session.add(new_profile)
    session.commit()
    return new_user

Чуть оптимизируем его

In [ ]:
@classmethod
@connection
def create(
    cls,
    username: str,
    password: str,
    email: str,
    phone=None,
    session: Session = None,
    **data,
):
    new_user = cls(username=username, password=password, email=email, phone=phone)
    session.add(new_user)
    session.flush()

    new_profile = Profile(user_id = new_user.id, **data)
    session.add(new_profile)
    session.commit()
    return new_user


- flush в SQLAlchemy отправляет изменения в базу данных без их окончательной фиксации, то есть без выполнения коммита. Это полезно, когда нужно сгенерировать данные, такие как идентификаторы (например, user.id), чтобы использовать их до фактического сохранения данных в базе. При этом сама транзакция остаётся открытой, и окончательное сохранение происходит позже, при вызове commit.

Почему это более оптимально:
- Работа с промежуточными данными: flush позволяет работать с данными, которые ещё не записаны в базу окончательно, но уже доступны для использования. Например, после создания пользователя вы можете получить его user.id и использовать его для добавления профиля, не выполняя коммит между этими операциями.

- Сокращение количества транзакций: Используя flush, мы избегаем нескольких коммитов. Это снижает нагрузку на базу данных, так как все изменения будут зафиксированы одним коммитом в конце транзакции. Таким образом, база данных фиксирует изменения только один раз, что ускоряет выполнение операций.

# Создание фэйковых данных

In [ ]:
import faker

fake = faker.Faker()
user_data = [
    {
        "username": fake.user_name(),
        "name": fake.name(),
        "email": fake.email(),
        "phone": fake.phone_number(),
    }
    for _ in range(10)
]
for data in user_data:
    print(data)

Кодом ниже можно сделать довавление сразу нескольких новых пользователей

In [ ]:
posts_data = [
    {
        "user_id": 7,
        "title": fake.street_title(),
        "contnet": fake.text(random.randint(100, 500)),
    }
    for _ in range(10)
]
print(*posts_data, sep="\n")

Post.create_many(posts_data)

In [ ]:
@classmethod
@connection
def create(
    cls,
    session: Session = None,
    **data,
):
    new_row = cls(**data)
    session.add(new_row)
    session.commit()
    return new_row

@classmethod
@connection
def create_many(
    cls,
    datas: list[dict],
    session: Session = None,
):
    new_rows = [cls(**data) for data in datas]
    session.add_all(new_rows)
    session.commit()
    return new_rows

Основные моменты:



1. Метод create: Этот метод позволяет добавить одну запись (например, одного пользователя) в базу данных.

   - Он принимает сессию базы данных и значения для полей записи в виде именованных аргументов (**values).

   - Создаётся новый экземпляр модели с переданными данными, затем он добавляется в сессию.

   - После этого вызывается commit, чтобы зафиксировать изменения в базе данных.

   - В случае ошибки, происходит откат (rollback), и ошибка выбрасывается.

2. Метод create_many: Этот метод используется для добавления сразу нескольких записей в базу данных за один раз.

    - Он принимает список словарей, где каждый словарь содержит данные для одной записи.

    - Из этих словарей создаются экземпляры модели и добавляются в сессию с помощью create_all.

    - После добавления всех экземпляров вызывается коммит для сохранения изменений.

    - Если возникает ошибка, как и в первом методе, вызывается откат транзакции и ошибка поднимается дальше.

# Обновление данных(update)
- Обновление каких либо данных ячейки столбца в БД

In [ ]:
@classmethod
@connection
def update(
    cls,
    id: int,
    session: Session = None,
    **data
):
    query = select(cls).where(cls.id == id)
    rows = session.execute(query)
    concrete_row = rows.scalar_one_or_none()
    
    if not concrete_row:
        raise ValueError(f'Данные с таким id в таблице {cls.__tablename__} не найдены')
    
    for key, value in data.items():
        if key not in concrete_row.__dict__:
            raise ValueError(f'Колонки "{key}" нету в таблице {cls.__tablename__}')
        if getattr(concrete_row, key) != value:
            setattr(concrete_row, key, value)
    
    session.commit()
    return concrete_row
            
            

Для того чтобы выполнять запрос в базу данных через сессию используется объект session а конкретно его метод execute

- execute(*запрос в бд*) - это специальная комманда которая может работать как с core так и ORM запросами. 
- В execute рекомендуется передавать переменную запроса а не сразу запрос, чтобы в дальнейшем по необходимости изменить тело запроса
- Для самих запросов(ORM) всегда используются функции-комманды самого SQL но в реализации через sqlalchemy, например select
- scalar - это специальная функция которая позволяет нам получить данные из запроса execute в читабельном для python виде. Вариаций scalar'ов около 5 видов.

  - rows.scalars() # Все содержимое в виде объекта со списком строк
  - rows.scalars().first() # Получить первый объект или None
  - rows.scalars().all() # Все содержимое в виде списка объектов-моделей
  - rows.scalar_one() # Получает 1 значение из таблицы если таких нет то ошибка
  - rows.scalar_one_or_none() # Получает 1 значение из таблицы если таких нет, то возвращает None

# Оптимизируем его 

In [ ]:
@classmethod
@connection
def update(
    cls,
    id: int,
    session: Session = None,
    **data
):
    query = select(cls).where(cls.id == id).with_for_update() # Основное изменение
    rows = session.execute(query)
    concrete_row = rows.scalar_one_or_none()

    
    if not concrete_row:
        raise ValueError(f'Данные с таким id в таблице {cls.__tablename__} не найдены')
    
    for key, value in data.items():
        if key not in concrete_row.__dict__:
            raise ValueError(f'Колонки "{key}" нету в таблице {cls.__tablename__}')
        if getattr(concrete_row, key) != value:
            setattr(concrete_row, key, value)
    
    session.commit()
    return concrete_row

with_for_update() - в конце запроса, блокирует строку таблицы до первого commit'a